In [1]:
import sqlite3
import pandas as pd
import numpy as np

import utils
from utils import TimedSQLiteConnection, get_query, get_query_one, get_query_dict, \
get_query_course_dict, get_query_df

conn = TimedSQLiteConnection('UMGC.db')
timedConnection = conn # so I can test function code directly

In [2]:
default_first_term = 'Spring 2024'
disabled_program_menu_items = {
        'Cybersecurity Technology',
        'Social Science',
        'Applied Technology',
        'Web and Digital Design',        
        'East Asian Studies',
        'English',
        'General Studies',
        'History'
    }
role = 'guest'
logged_in = False

In [3]:
# Settled Functions

# parallel the utils.get_required_program_courses code
async def get_required_program_courses_no_q(timedConnection, student_info):
    program_id = student_info['program_id']
    query = '''
        SELECT 
            id,
            course, 
            course_type as type,
            title,
            credits,
            pre,
            pre_credits,
            substitutions,
            description
        FROM program_requirements_view
        WHERE program_id = ?
    '''
    df = await get_query_df(timedConnection, query, params=(program_id,))
    return df


In [4]:
student_info = utils.initialize_student_info()
student_info['ge'] = utils.initialize_ge() # only if undergraduate
student_info_populated = False # may be needed later 
student_data = utils.initialize_student_data() # will have required, periods, schedule

student_info['user_id'] = 0
student_info

# Step 1. Student name registered

# from utils.set_user_vars_given_role when retrieving from database
# here it is entered and registered
student_info['name'] = 'Joe Smith'

student_info['app_stage_id'] = 1
student_info['app_stage'] = 'new' # get this from query

student_info
# Step 2. Student information

student_info['financial_aid'] = 1
student_info['transfer_credits'] = 0
student_info['student_profile'] = 'Full-time' # check spelling
student_info['resident_status'] = 'In-State' # check spelling
student_info['first_term'] = 'Spring 2024'

student_info['app_stage_id'] = 2
student_info['app_stage'] = 'personalized' # get this from query

student_info
# Step 3. Choose program

student_info['menu']['degree'] = 2
student_info['menu']['area_of_study'] = 1
student_info['menu']['program'] = 2
student_info['program_id'] = 2
student_info['degree_program'] = 'BS in Accounting'

student_info['app_stage_id'] = 3
student_info['app_stage'] = 'program chosen' # get this from query

## Build a program from scratch
### Step 1: Get program course list (as df?)
### Step 2: Get GE course list (as df?)
### Step 3: Append
- If no 'Required,General' in `program_df`, simply append GE to Program
- If 'Required,General' in `program_df`, categorize them as GE and merge

In [ ]:
# Once program is chosen, fill in GE information including any GE 

In [29]:
# Build the Program course list

# Task 1. Get the required classes from the program
required_df = await get_required_program_courses_no_q(timedConnection, student_info)

In [31]:
# Build the GE course list
# Create a blank list of GE course requirements with all courses set to 'GENERAL'
query = '''
    SELECT id AS ge_id, 'GENERAL' AS course, requirement, abbr, part, credits, 
        'ge_' || abbr || '_' || part AS course_slot 
    FROM general_education_requirements 
'''
ge_course_list = await get_query_dict(timedConnection, query)

# update with information from student_info['ge']
ge_course_list = utils.update_ge_list_from_student_info(ge_course_list, student_info)

# Update the ge_course_list 
ge_course_list, elective_data = await utils.build_program_course_list(required_df, timedConnection, ge_course_list)

In [32]:
elective_data

[]

In [48]:
# Process ge_course_list_df
ge_course_list_df = pd.DataFrame(ge_course_list)
ge_course_list_df = utils.update_bio_df(ge_course_list_df)
ge_course_list_df['type'] = 'General'
ge_course_list_df = ge_course_list_df.rename(columns={'course_slot': 'ge'})

# Merge possible GE (should only work if type = 'Required,General' )
course_list_df = required_df[['id', 'course', 'type', 'credits']].copy()
merged_df = pd.merge(course_list_df, ge_course_list_df[['ge_id', 'course', 'ge']], on='course')
merged_df

,id,course,type,credits,ge_id,ge
0,13,ECON 201,"Required,General",3,12,ge_beh_1
1,14,ECON 203,"Required,General",3,13,ge_beh_2
2,15,STAT 200,"Required,General",3,5,ge_math_1
3,16,WRTG 293,"Required,General",3,3,ge_comm_3


In [49]:
# concatenate original and 'Required,General' 
course_list_df = pd.concat(
    [course_list_df[course_list_df['type'] != 'Required,General'], merged_df], ignore_index=True)


,id,course,type,credits,ge_id,ge
0,1,ACCT 220,Major,3,NaN,NaN
1,2,ACCT 221,Major,3,NaN,NaN
2,3,ACCT 310,Major,3,NaN,NaN
3,4,ACCT 311,Major,3,NaN,NaN
4,5,ACCT 321,Major,3,NaN,NaN
5,6,ACCT 323,Major,3,NaN,NaN
6,7,ACCT 326,Major,3,NaN,NaN
7,8,ACCT 410,Major,3,NaN,NaN
8,9,ACCT 422,Major,3,NaN,NaN
9,10,ACCT 424,Major,3,NaN,NaN


In [39]:
course_list_df['ge_id'] = np.nan
course_list_df['ge'] = ''
if len(merged_df) > 0:
    # Remove those rows from required_df
    

4

In [33]:
df1 = program_df[['id', 'course', 'type', 'credits' ]].copy()
df1['ge'] = ''
df1.head()

,id,course,type,credits,ge
0,1,ACCT 220,Major,3,
1,2,ACCT 221,Major,3,
2,3,ACCT 310,Major,3,
3,4,ACCT 311,Major,3,
4,5,ACCT 321,Major,3,


In [11]:
df2 = ge_course_list_df[['course', 'type', 'credits', 'ge']].copy()
# Remove rows from df2 that have courses already in df1
df2 = df2[~df2['course'].isin(df1['course'])]
df2

,course,type,credits,ge
0,WRTG 111,General,3,ge_comm_1
1,WRTG 112,General,3,ge_comm_2
3,GENERAL,General,3,ge_comm_4
5,GENERAL,General,3,ge_arts_1
6,GENERAL,General,3,ge_arts_2
7,GENERAL,General,4,ge_bio_1
10,GENERAL,General,3,ge_bio_2
13,GENERAL,General,3,ge_res_1
14,LIBS 150,General,1,ge_res_2
15,GENERAL,General,3,ge_res_3


,id,course,type,credits,ge_id,ge
0,13,ECON 201,"Required,General",3,12,ge_beh_1
1,14,ECON 203,"Required,General",3,13,ge_beh_2
2,15,STAT 200,"Required,General",3,5,ge_math_1
3,16,WRTG 293,"Required,General",3,3,ge_comm_3


In [28]:
ge_course_list_df[~ge_course_list_df['ge_id'].isin(merged_df['ge_id'].unique())][['course', 'type', 'credits', 'ge_id', 'ge']]

,course,type,credits,ge_id,ge
0,WRTG 111,General,3,1,ge_comm_1
1,WRTG 112,General,3,2,ge_comm_2
3,GENERAL,General,3,4,ge_comm_4
5,GENERAL,General,3,6,ge_arts_1
6,GENERAL,General,3,7,ge_arts_2
7,GENERAL,General,4,8,ge_bio_1
10,GENERAL,General,3,11,ge_bio_2
13,GENERAL,General,3,14,ge_res_1
14,LIBS 150,General,1,15,ge_res_2
15,GENERAL,General,3,16,ge_res_3


In [21]:
filtered_df = program_df[program_df['type'] != 'Required,General'][['id','course','type', 'credits']]
filtered_df['ge_id'] = np.nan
filtered_df['ge'] = ''
filtered_df

,id,course,type,credits,ge_id,ge
0,1,ACCT 220,Major,3,NaN,
1,2,ACCT 221,Major,3,NaN,
2,3,ACCT 310,Major,3,NaN,
3,4,ACCT 311,Major,3,NaN,
4,5,ACCT 321,Major,3,NaN,
5,6,ACCT 323,Major,3,NaN,
6,7,ACCT 326,Major,3,NaN,
7,8,ACCT 410,Major,3,NaN,
8,9,ACCT 422,Major,3,NaN,
9,10,ACCT 424,Major,3,NaN,


In [ ]:

df2 = ge_course_list_df[['course', 'type', 'credits', 'ge']].copy

#df2_filtered = df2[~df2['course'].isin(df1['course'])]
#df2_filtered

In [29]:
df2['course']

TypeError: 'method' object is not subscriptable

In [50]:
# Step 1: Filter df to get bio_df
bio_df = df[(df['course'] == 'GENERAL') & (df['course_slot'].str.contains('bio_1[ab]'))]
bio_df


,ge_id,course,requirement,abbr,part,credits,course_slot
7,8,GENERAL,4,bio,1a,4,ge_bio_1a
8,9,GENERAL,4,bio,1b,4,ge_bio_1b


In [51]:
len(bio_df)

2

In [44]:
first_row = bio_df.iloc[0]
df.loc[df['ge_id'] == first_row['ge_id'], 'part'] = '1'
df.loc[df['ge_id'] == first_row['ge_id'], 'course_slot'] = 'ge_bio_1'


In [45]:
df

,ge_id,course,requirement,abbr,part,credits,course_slot
0,1,WRTG 111,1,comm,1,3,ge_comm_1
1,2,WRTG 112,1,comm,2,3,ge_comm_2
2,3,WRTG 293,1,comm,3,3,ge_comm_3
3,4,GENERAL,1,comm,4,3,ge_comm_4
4,5,STAT 200,2,math,1,3,ge_math_1
5,6,GENERAL,3,arts,1,3,ge_arts_1
6,7,GENERAL,3,arts,2,3,ge_arts_2
7,8,GENERAL,4,bio,1,4,ge_bio_1
8,9,GENERAL,4,bio,1b,4,ge_bio_1b
9,10,GENERAL,4,bio,1c,4,ge_bio_1c


In [52]:
df = df[~df['ge_id'].isin(bio_df['ge_id'])]
df

,ge_id,course,requirement,abbr,part,credits,course_slot
0,1,WRTG 111,1,comm,1,3,ge_comm_1
1,2,WRTG 112,1,comm,2,3,ge_comm_2
2,3,WRTG 293,1,comm,3,3,ge_comm_3
3,4,GENERAL,1,comm,4,3,ge_comm_4
4,5,STAT 200,2,math,1,3,ge_math_1
5,6,GENERAL,3,arts,1,3,ge_arts_1
6,7,GENERAL,3,arts,2,3,ge_arts_2
9,10,GENERAL,4,bio,1c,4,ge_bio_1c
10,11,GENERAL,4,bio,2,3,ge_bio_2
11,12,ECON 201,5,beh,1,3,ge_beh_1


In [53]:
df.loc[df['part'].str.contains('1[abc]'), 'part'] = '1'
df.loc[df['course_slot'].str.contains('ge_bio_1[abc]'), 'course_slot'] = 'ge_bio_1'
df

,ge_id,course,requirement,abbr,part,credits,course_slot
0,1,WRTG 111,1,comm,1,3,ge_comm_1
1,2,WRTG 112,1,comm,2,3,ge_comm_2
2,3,WRTG 293,1,comm,3,3,ge_comm_3
3,4,GENERAL,1,comm,4,3,ge_comm_4
4,5,STAT 200,2,math,1,3,ge_math_1
5,6,GENERAL,3,arts,1,3,ge_arts_1
6,7,GENERAL,3,arts,2,3,ge_arts_2
9,10,GENERAL,4,bio,1,4,ge_bio_1
10,11,GENERAL,4,bio,2,3,ge_bio_2
11,12,ECON 201,5,beh,1,3,ge_beh_1


In [38]:
# Step 2: Check the length of bio_df
if len(bio_df) == 3:
    # Take the first row and update 'part' and 'course_slot' in the original df
    first_row = bio_df.iloc[0]
    df.loc[df['ge_id'] == first_row['ge_id'], 'part'] = '1'
    df.loc[df['ge_id'] == first_row['ge_id'], 'course_slot'] = 'ge_bio_1'
    
    # Remove the other two rows from df
    df = df[~df['ge_id'].isin(bio_df.iloc[1:]['ge_id'])]
    
elif len(bio_df) == 2:
    # Delete rows from df identified by their 'ge_id'
    df = df[~df['ge_id'].isin(bio_df['ge_id'])]
else:
    # this should raise an error
    # add this later
    pass

# Step 3: Change 'part' and 'course_slot' in df
df.loc[df['part'].str.contains('1[abc]'), 'part'] = '1'
df.loc[df['course_slot'].str.contains('ge_bio_1[abc]'), 'course_slot'] = 'ge_bio_1'


,ge_id,course,requirement,abbr,part,credits,course_slot
7,8,GENERAL,4,bio,1a,4,ge_bio_1a
8,9,GENERAL,4,bio,1b,4,ge_bio_1b
9,10,GENERAL,4,bio,1c,4,ge_bio_1c


In [30]:
count_done_bio = len(df[(df['course'] != 'GENERAL') & (df['course_slot'].str.contains('bio_1'))])
count_done_bio

# if count_done_bio = 0, then delete 

0

In [21]:
# Check if any of the abbr='bio' and part='1[abc]' are not 'GENERAL'
bio_non_general = any(course['course'] != 'GENERAL' for course in ge_remaining_list if course['abbr'] == 'bio' and course['part'] in ['1a', '1b', '1c'])
bio_non_general

False

In [ ]:

# Filter out the records with abbr='bio' and part='1[abc]'
bio_records = [course for course in ge_course_list if course['abbr'] == 'bio' and course['part'] in ['1a', '1b', '1c']]

# Check if all three parts are 'GENERAL'
all_general = all(course['course'] == 'GENERAL' for course in bio_records)

# If all three parts are 'GENERAL', rename part='1a' to part='1' and omit '1b' and '1c'
if all_general:
    # Rename part='1a' to part='1'
    for course in ge_course_list:
        if course['abbr'] == 'bio' and course['part'] == '1a':
            course['part'] = '1'
    # Filter out '1b' and '1c' records
    ge_course_list = [course for course in ge_course_list if not (course['abbr'] == 'bio' and course['part'] in ['1b', '1c'])]
else:
    # If one of the parts is not 'GENERAL', find the non-'GENERAL' part and rename it to '1'
    for course in bio_records:
        if course['course'] != 'GENERAL':
            course['part'] = '1'
            # Filter out '1a', '1b', and '1c' records except the non-'GENERAL' part
            ge_course_list = [c for c in ge_course_list if not (c['abbr'] == 'bio' and c['part'] != course['part'])]

# Filter out the records with course='GENERAL'
general_courses = [course for course in ge_course_list if course['course'] == 'GENERAL']


In [17]:
pd.DataFrame(ge_course_list)
df[df['course'] == 'GENERAL']

,ge_id,course,requirement,abbr,part,credits,course_slot
3,4,GENERAL,1,comm,4,3,ge_comm_4
5,6,GENERAL,3,arts,1,3,ge_arts_1
6,7,GENERAL,3,arts,2,3,ge_arts_2
7,8,GENERAL,4,bio,1a,4,ge_bio_1a
8,9,GENERAL,4,bio,1b,4,ge_bio_1b
9,10,GENERAL,4,bio,1c,4,ge_bio_1c
10,11,GENERAL,4,bio,2,3,ge_bio_2
13,14,GENERAL,6,res,1,3,ge_res_1
15,16,GENERAL,6,res,3,3,ge_res_3


In [11]:
ge_course_list

[{'ge_id': 1,
  'course': 'WRTG 111',
  'requirement': 1,
  'abbr': 'comm',
  'part': '1',
  'credits': 3,
  'course_slot': 'ge_comm_1'},
 {'ge_id': 2,
  'course': 'WRTG 112',
  'requirement': 1,
  'abbr': 'comm',
  'part': '2',
  'credits': 3,
  'course_slot': 'ge_comm_2'},
 {'ge_id': 3,
  'course': 'WRTG 293',
  'requirement': 1,
  'abbr': 'comm',
  'part': '3',
  'credits': 3,
  'course_slot': 'ge_comm_3'},
 {'ge_id': 4,
  'course': 'GENERAL',
  'requirement': 1,
  'abbr': 'comm',
  'part': '4',
  'credits': 3,
  'course_slot': 'ge_comm_4'},
 {'ge_id': 5,
  'course': 'STAT 200',
  'requirement': 2,
  'abbr': 'math',
  'part': '1',
  'credits': 3,
  'course_slot': 'ge_math_1'},
 {'ge_id': 6,
  'course': 'GENERAL',
  'requirement': 3,
  'abbr': 'arts',
  'part': '1',
  'credits': 3,
  'course_slot': 'ge_arts_1'},
 {'ge_id': 7,
  'course': 'GENERAL',
  'requirement': 3,
  'abbr': 'arts',
  'part': '2',
  'credits': 3,
  'course_slot': 'ge_arts_2'},
 {'ge_id': 8,
  'course': 'GENERAL',


In [13]:
row = ge_course_list[2]
row

{'ge_id': 3,
 'course': 'WRTG 293',
 'requirement': 1,
 'abbr': 'comm',
 'part': '3',
 'credits': 3,
 'course_slot': 'ge_comm_3'}

In [16]:
pd.DataFrame(ge_course_list)

,ge_id,course,requirement,abbr,part,credits,course_slot
0,1,WRTG 111,1,comm,1,3,ge_comm_1
1,2,WRTG 112,1,comm,2,3,ge_comm_2
2,3,WRTG 293,1,comm,3,3,ge_comm_3
3,4,GENERAL,1,comm,4,3,ge_comm_4
4,5,STAT 200,2,math,1,3,ge_math_1
5,6,GENERAL,3,arts,1,3,ge_arts_1
6,7,GENERAL,3,arts,2,3,ge_arts_2
7,8,GENERAL,4,bio,1a,4,ge_bio_1a
8,9,GENERAL,4,bio,1b,4,ge_bio_1b
9,10,GENERAL,4,bio,1c,4,ge_bio_1c


In [15]:
student_info = update_student_info_ge(student_info, ge_course_list)
student_info

{'user_id': 0,
 'name': 'Joe Smith',
 'financial_aid': 1,
 'resident_status': 'In-State',
 'student_profile': 'Full-time',
 'transfer_credits': 0,
 'app_stage': 'program chosen',
 'app_stage_id': 3,
 'first_term': 'Spring 2024',
 'program_id': 2,
 'degree_program': 'BS in Accounting',
 'menu': {'degree': 2, 'area_of_study': 1, 'program': 2},
 'ge': {'arts': {'1': None, '2': None, 'nopre': False},
  'beh': {'1': 'ECON 201', '2': 'ECON 203', 'nopre': False},
  'bio': {'1a': None, '1b': None, '1c': None, '2': None, 'nopre': False},
  'comm': {'1': 'WRTG 111',
   '2': 'WRTG 112',
   '3': 'WRTG 293',
   '4': None,
   'nopre': False},
  'math': {'1': 'STAT 200', 'nopre': False},
  'res': {'1': None,
   '2': 'LIBS 150',
   '3': None,
   '3a': None,
   '3b': None,
   '3c': None,
   'nopre': False}}}

In [10]:
# Now fill in the rest of the GE requirements
student_info['ge']

{'arts': {'1': None, '2': None, 'nopre': False},
 'beh': {'1': None, '2': None, 'nopre': False},
 'bio': {'1a': None, '1b': None, '1c': None, '2': None, 'nopre': False},
 'comm': {'1': 'WRTG 111',
  '2': 'WRTG 112',
  '3': None,
  '4': None,
  'nopre': False},
 'math': {'1': None, 'nopre': False},
 'res': {'1': None,
  '2': 'LIBS 150',
  '3': None,
  '3a': None,
  '3b': None,
  '3c': None,
  'nopre': False}}

In [ ]:
program_query = '''
    SELECT program_id AS name, program_name AS label, disabled
    FROM menu_all_view 
    WHERE menu_degree_id = ? AND menu_area_id = ?
'''

In [ ]:
rows = await get_query(timedConnection, program_query, (1, 7))
[(row['name'], row['label'], bool(row['disabled']) ) for row in rows]

In [ ]:
choices = await utils.get_choices_with_disabled(timedConnection, program_query, (1, 7))
choices

In [ ]:
# replicate code from get_required_program_courses(q)
async def get_required_df(program_id, timedConnection = conn):
    query = '''
        SELECT 
            id,
            course, 
            course_type as type,
            title,
            credits,
            pre,
            pre_credits,
            substitutions,
            description
        FROM program_requirements_view
        WHERE program_id = ?
    '''
    df = await get_query_df(timedConnection, query, params=(program_id,))
    return df

program_id = 2
df = await get_required_df(program_id)
df.head()

df[df['type'] == 'Required,General']

# Step 0: Convert required df to a dictionary
dict_list = df.to_dict(orient='records')

# Step 1: Extract the required courses
required_courses = [row['course'] for row in dict_list if row['type'] == 'Required,General']
required = tuple(required_courses)
required

# Step 2: Construct the SQL query
# Using placeholders for the IN clause
query = f'''
    SELECT ge_id, course 
    FROM ge_view 
    WHERE course IN ({','.join(['?' for _ in required_courses])})
'''

# Step 3: Execute the query
result = await get_query_dict(timedConnection, query, params=required)
result

In [ ]:
result

In [ ]:
#old_id: new_id
ge_map = [
    { 'old_id': 14, 'new_id': 16, 'abbr': 'res'},
    { 'old_id': 13, 'new_id': 15, 'abbr': 'res'},
    { 'old_id': 12, 'new_id': 14, 'abbr': 'res'},
    { 'old_id': 11, 'new_id': 13, 'abbr': 'beh'},
    { 'old_id': 11, 'new_id': 12, 'abbr': 'beh'},
    { 'old_id': 10, 'new_id': 11, 'abbr': 'bio'},
    { 'old_id':  9, 'new_id': 10, 'abbr': 'bio'},
    { 'old_id':  8, 'new_id':  9, 'abbr': 'bio'},
    { 'old_id':  7, 'new_id':  8, 'abbr': 'bio'},
    { 'old_id':  6, 'new_id':  7, 'abbr': 'arts'},
    { 'old_id':  6, 'new_id':  6, 'abbr': 'arts'},
    { 'old_id':  5, 'new_id':  5, 'abbr': 'math'},
    { 'old_id':  4, 'new_id':  4, 'abbr': 'comm'},
    { 'old_id':  3, 'new_id':  3, 'abbr': 'comm'},
    { 'old_id':  2, 'new_id':  2, 'abbr': 'comm'},
    { 'old_id':  1, 'new_id':  1, 'abbr': 'comm'}
    ]

In [ ]:
dict_list

In [ ]:
courses = [{'course': row['course'], 'credits': row['credits'], 
            'ge': '', 'type': row['type']}  for row in dict_list]
courses

In [ ]:



req_ge = [row['course'] for row in courses if row['type'] == 'Required,General']
req_ge

SELECT ge_id, course
FROM ge_view 
WHERE course IN ('ECON 201', 'ECON 203', 'IFSM 300', 'STAT 200')


In [ ]:
student_info = utils.initialize_student_info()
student_info

In [ ]:
student_data = utils.initialize_student_data()
student_data

In [ ]:
user_id = 3
student_info['user_id'] = user_id
student_data['user_id'] = user_id

In [ ]:
query = 'SELECT * FROM student_info_view WHERE user_id = ?'
row = await get_query_one(timedConnection, query, params=(user_id,))


In [ ]:
attributes = ['resident_status', 'app_stage_id', 'app_stage', 'student_profile', 'financial_aid', 
              'transfer_credits', 'program_id']
student_info.update({name: row[name] for name in attributes})
student_info

In [ ]:
row = await utils.get_program_title(timedConnection, student_info['program_id'])
if row:
    student_info['degree_program'] = row['title']
    student_info['degree_id'] = row['id']
student_info

In [ ]:
student_data

In [ ]:
query = 'SELECT * FROM student_progress_d3_view WHERE user_id = ?'
# note: 'course' is named 'name' in student_progress_d3_view 
df = await get_query_df(conn, query, params=(student_info['user_id'],))
student_data['schedule'] = df

In [ ]:
html_template = utils.create_html_template(df, 'SPRING 2024')


In [ ]:
df_input = df.copy()
df_input.rename(columns={'term': 'period'}, inplace=True)
df_display, headers_display = utils.prepare_d3_data(df_input)

In [ ]:
df_display

In [ ]:
periods = utils.generate_periods(summer=True)


In [ ]:
def parse_prerequisites(prereq_string):
    '''
    Parse the prerequisites string and return a list of prerequisite groups.
    Each group contains lists of courses that satisfy the 'or' and 'and' patterns.
    '''
    import re
    prereq_string = prereq_string.strip()
    prereq_string = re.sub(r'\s+', ' ', prereq_string)
    prereq_string = re.sub(r'\(', '', prereq_string)
    prereq_string = re.sub(r'\)', '', prereq_string)
    
    and_groups = prereq_string.split('&')
    prereq_groups = [group.split('|') for group in and_groups]
    
    # Strip whitespace from each course in the groups
    prereq_groups = [[course.strip() for course in group] for group in prereq_groups]
    
    return prereq_groups


In [ ]:
parse_prerequisites('(CMIT 202 | CMIT 265)')

In [ ]:
parse_prerequisites('(STAT 200 & (MATH 115 | MATH 108))')

# Keycloak Stuff

In [ ]:
dir(keycloak)

In [ ]:
import keycloak

In [ ]:
#from keycloak import KeycloakOpenID
#from keycloak.exceptions import KeycloakAuthenticationError

In [ ]:
keycloak

In [ ]:
# Configuration
keycloak_server_url = "http://localhost:8080/auth/"
keycloak_realm_name = "your-realm"
keycloak_client_id = "your-client-id"
keycloak_client_secret = "your-client-secret"  # If using a confidential client
keycloak_redirect_uri = "http://localhost:8888/callback"
keycloak_username = "sample-user"
keycloak_password = "sample-password"

# Initialize Keycloak client
keycloak_openid = KeycloakOpenID(server_url=keycloak_server_url,
                                 client_id=keycloak_client_id,
                                 realm_name=keycloak_realm_name,
                                 client_secret_key=keycloak_client_secret)

# Authenticate and obtain access token
try:
    token = keycloak_openid.token(keycloak_username, keycloak_password)
    access_token = token['access_token']
    print("Access token:", access_token)
except KeycloakAuthenticationError as e:
    print(f"Failed to authenticate: {e}")

# Fetch user information
if access_token:
    userinfo = keycloak_openid.userinfo(access_token)
    print("User information:", userinfo)

In [ ]:
'  My Name'.strip()

In [ ]:
timedConnection = conn
program_id = 5
user_id = 3
student_data = {}

In [ ]:
query = 'SELECT * FROM student_progress_d3_view WHERE user_id = ?'
df = await get_query_df(timedConnection, query, params=(user_id,))
student_data['schedule'] = df
df

In [ ]:
query = '''
        SELECT 
            id,
            course, 
            course_type as type,
            title,
            credits,
            pre,
            pre_credits,
            substitutions,
            description
        FROM program_requirements_view
        WHERE program_id = ?
'''
df = await get_query_df(timedConnection, query, params=(program_id,))
student_data['required'] = df
df

In [ ]:
student_data

In [ ]:
attributes = ['resident_status', 'app_stage_id', 'app_stage', 'student_profile', 'financial_aid', 
        'transfer_credits', 'program_id']

In [ ]:
attributes

In [ ]:
student_info

In [ ]:
aa = 'ge_bio_1a'

In [ ]:
aa.split('_')

In [ ]:
aa.split('_')[2]